## Composite Estimators using Pipeline & FeatureUnions

<hr>

### Agenda
1. Introduction to Composite Estimators
2. Pipelines
3. TransformedTargetRegressor
4. FeatureUnions
5. ColumnTransformer
6. GridSearch on pipeline

PS: scikit version 0.20

<hr>

### 1. Introduction to Composite Estimators
* One or more transformers are connected to estimators resulting into composite estimator.
* Composite transformer is implemented using Pipeline
* FeatureUnion is used to concatenate output of transformers to create derived feature
* Pipeline make machine learning code reuseable & modular

### 2. Pipeline
* Before data is fed to learning algorithm, it needs to be handled for missing values.
* Different pre-processing needs to be done.
* The output of preprocessor is to be subjected to next preprocessor & finally the estimator
* This whole process can be automated using Pipeline

<img src="https://github.com/awantik/machine-learning-slides/blob/master/pipeline-ml2.png?raw=true">

* Intermediate steps .i.e transformers must implement fit & transform
* The same trained pipeline can used for prediction

#### Predicting horror author from text 

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving horror-train.csv to horror-train.csv
User uploaded file "horror-train.csv" with length 3295644 bytes


In [ ]:
horror_train_data = pd.read_csv('horror-train.csv')

In [ ]:
horror_train_data.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving horror-test.csv to horror-test.csv
User uploaded file "horror-test.csv" with length 1351241 bytes


In [ ]:
horror_test_data= pd.read_csv('horror-test.csv')

In [ ]:
horror_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8392 entries, 0 to 8391
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8392 non-null   object
 1   text    8392 non-null   object
dtypes: object(2)
memory usage: 131.2+ KB


In [ ]:
horror_train_data = horror_train_data[['text','author']]

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
pipelines = []
for model in [LogisticRegression(), DecisionTreeClassifier(), MultinomialNB(), SVC()]:
    pipeline = make_pipeline(
              CountVectorizer(stop_words='english'),
              TfidfTransformer(),
              model)
    pipelines.append(pipeline)

In [ ]:
pipelines[3].steps[2]

('svc',
 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False))

In [ ]:
pipelines[2].steps[1]

('tfidftransformer',
 TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
trainX,testX,trainY,testY = train_test_split(horror_train_data.text, horror_train_data.author)

In [ ]:
for pipeline in pipelines:
    pipeline.fit(trainX, trainY)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
for pipeline in pipelines:
    print (pipeline.score(testX, testY))

0.7928498467824311
0.5873340143003064
0.8081716036772216
0.7842696629213484


In [ ]:
horror_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8392 entries, 0 to 8391
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8392 non-null   object
 1   text    8392 non-null   object
dtypes: object(2)
memory usage: 131.2+ KB


In [ ]:
results = []
for pipeline in pipelines:
    result = pipeline.predict(horror_test_data.text)
    results.append(result)

In [ ]:
results

[array(['MWS', 'EAP', 'EAP', ..., 'EAP', 'MWS', 'HPL'], dtype=object),
 array(['MWS', 'EAP', 'HPL', ..., 'EAP', 'MWS', 'EAP'], dtype=object),
 array(['MWS', 'EAP', 'EAP', ..., 'EAP', 'MWS', 'HPL'], dtype='<U3'),
 array(['MWS', 'EAP', 'EAP', ..., 'EAP', 'MWS', 'HPL'], dtype=object)]

In [ ]:
pipelines[0].steps[0][1].transform(horror_test_data.text)

<8392x22210 sparse matrix of type '<class 'numpy.int64'>'
	with 88740 stored elements in Compressed Sparse Row format>

#### Caching transformers within a Pipeline
* Storing state of transformers is also possible to prevent recomputation of transformers
* When pipeline is subjected to GridSearch situations like this happens

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
svc_pipe =  make_pipeline(
              CountVectorizer(stop_words='english'),
              TfidfTransformer(),
              SVC())

In [ ]:
dt_pipe = make_pipeline(
              CountVectorizer(stop_words='english'),
              TfidfTransformer(),
              DecisionTreeClassifier())

In [ ]:
svc_pipe

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=Non...one)),
                ('tfidftransformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='

In [ ]:
svc_pipe.steps[2]

('svc',
 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False))

In [ ]:
import numpy as np
params = {
    'svc__C': list(np.logspace(1,20,20))
}

In [ ]:
dt_pipe.steps[2]

('decisiontreeclassifier',
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'))

In [ ]:
params = {
    'countvectorizer__max_features':[5000,7500,10000],
    'decisiontreeclassifier__max_depth':[100,200]
}

In [ ]:
gs = GridSearchCV(dt_pipe,cv=5,param_grid=params, n_jobs=-1)

In [ ]:
%timeit gs.fit(trainX,trainY)

1 loop, best of 5: 50.2 s per loop


In [ ]:
gs.best_params_

{'countvectorizer__max_features': 10000,
 'decisiontreeclassifier__max_depth': 200}

In [ ]:
gs.best_score_

0.6008577029518155

%timeit gs.fit(trainX,trainY)

In [ ]:
! pip install sklearn.utils

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for sklearn.utils: filename=sklearn_utils-0.0.15-cp36-none-any.whl size=27880 sha256=4dfef61f26c59e53c5d96bd109fa9e2c3e1b31dcb11b4dc935fa77d3ecac6335
  Stored in directory: /root/.cache/pip/wheels/eb/4e/57/de4e188f2e7989e7e45e07af4fea41237ae77b9931f3c82c26
  Created wheel for dill: filename=dill-0.2.7.1-cp36-none-any.whl size=54544 sha256=d17ec4381fbaa3a6c01a2c56ad87f50c9ea9451c1a2e019546750551cb9d6009
  Stored in directory: /root/.cache/pip/wheels/99/c4/ed/1b64d2d5809e60d5a3685530432f6159d6a9959739facb61f2
Successfully built sklearn.utils dill
ERROR: multiprocess 0.70.9 has requirement dill>=0.3.1, but you'll have dill 0.2.7.1 which is incompatible.
  Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1


In [ ]:
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.utils import Memory

cachedir = mkdtemp()
memory = Memory(location=cachedir, verbose=0)
svc_pipe_cached =  make_pipeline(
              CountVectorizer(stop_words='english'),
              TfidfTransformer(),
              SVC(), memory = memory)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Memory is deprecated; deprecated in version 0.20.1 to be removed in version 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
! pip install joblib
joblib.memory

In [ ]:
params = {
    'svc__C': list(np.logspace(1,20,20))
}

In [ ]:
gs_cached = GridSearchCV(svc_pipe_cached,cv=5,param_grid=params, verbose=0)

In [ ]:
%timeit gs_cached.fit(trainX,trainY)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:317: UserWarning: Persisting input arguments took 2.00s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:317: UserWarning: Persisting input arguments took 2.01s to run.
If this happens often in your code, it can cause performance problems 
(results will b

KeyboardInterrupt: ignored

### 3. Transforming target in regression
* Dependent variables & independent variables should be linearly related
* In case, dependent variable is not normally distribted. We can make it happen for better error.
* The prediction also needs to be remapped
* This entire process can be automated using TransformedTargetRegressor

In [ ]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
boston = load_boston()

In [ ]:
X = boston.data

In [ ]:
y = boston.target

In [ ]:
regressor = LinearRegression()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print('R2 score: {0:.2f}'.format(regressor.score(X_test, y_test)))

R2 score: 0.64


In [ ]:
pred = regressor.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
mean_absolute_error(y_pred=pred, y_true=y_test)

3.6683301481357256

In [ ]:
r2_score(y_pred=pred,y_true=y_test )

0.635463843320211

### Convert data from non-normal distribution to normal distribution

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
# pt = PowerTransformer()

In [ ]:
qt = QuantileTransformer(output_distribution='normal')

In [ ]:
#X_tf = pt.fit_transform(X)
#OR
X_tf = qt.fit_transform(X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (506). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, random_state=0)

In [ ]:
regressor = LinearRegression()

In [ ]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print('R2 score: {0:.2f}'.format(regressor.score(X_test, y_test)))

R2 score: 0.66


In [ ]:
pred = regressor.predict(X_test)

In [ ]:
mean_absolute_error(y_pred=pred, y_true=y_test)

3.6327621516037127

In [ ]:
from sklearn.compose import TransformedTargetRegressor

In [ ]:
regr = TransformedTargetRegressor(regressor=regressor,transformer=qt)

In [ ]:
regr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2357: UserWarning: n_quantiles (1000) is greater than the total number of samples (379). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=LinearRegression(copy_X=True,
                                                      fit_intercept=True,
                                                      n_jobs=None,
                                                      normalize=False),
                           transformer=QuantileTransformer(copy=True,
                                                           ignore_implicit_zeros=False,
                                                           n_quantiles=1000,
                                                           output_distribution='normal',
                                                           random_state=None,
                                                           subsample=100000))

In [ ]:
pred = regr.predict(X_test)

In [ ]:
mean_absolute_error(y_pred=pred, y_true=y_test)

3.349020126293933

In [ ]:
r2_score(y_pred=pred, y_true=y_test)

0.7099444634911461

#### Hyper-parameters of TransformedTargetRegressor
* regressor - initialized model
* transformer - which supports transform & inverse_transform functions
* function - to convert target 
* inverse_function - to convert back predicted target in original data scale